In [1]:
def parse_input(filename):
    # Reads the input file and separates map lines from move lines.
    with open(filename, 'r') as f:
        lines = f.read().splitlines()

    # Identify the map lines (until first blank line or line of moves)
    # The puzzle states the map is shown first, then moves.
    # We'll assume a blank line or a line without '#' marks the start of moves.
    # If your input doesn't have a blank line, adjust accordingly.

    # Find where the map ends and moves begin
    # The warehouse is typically surrounded by walls (#), so we can detect move lines by absence of '#'.
    # If that assumption doesn't hold, you may need a different heuristic.

    # Let's assume the map lines all contain '#' at some point.
    # Once we reach a line that does not have '#' or is empty, that should be moves.

    warehouse_lines = []
    move_lines = []
    found_moves = False
    for line in lines:
        if '#' in line:
            if not found_moves:
                warehouse_lines.append(line)
            else:
                # Once we've started moves, '#' shouldn't appear again,
                # but if it does, that means input format might differ.
                # We assume standard format.
                move_lines.append(line)
        else:
            # No '#' in line, likely moves
            found_moves = True
            if line.strip():
                move_lines.append(line)

    return warehouse_lines, move_lines

def find_robot_and_boxes(grid):
    robot_pos = None
    boxes = []
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == '@':
                robot_pos = (r, c)
            elif grid[r][c] == 'O':
                boxes.append((r, c))
    return robot_pos, set(boxes)

def can_push_boxes(grid, boxes, start_r, start_c, dr, dc):
    # Attempt to push the chain of boxes starting at (start_r, start_c) in direction (dr, dc).
    # This may involve multiple boxes lined up. Check if we can move them all by one cell.
    # Return True if can push, along with updated box positions, else False.

    # Gather all boxes in a line in that direction.
    # Actually, we only need to consider pushing the boxes in front of the robot.
    # Check how far the chain goes:
    chain = []
    r, c = start_r, start_c
    while (r, c) in boxes:
        chain.append((r, c))
        r += dr
        c += dc

    # The next cell after the last box in the chain:
    target_r, target_c = r, c

    # If the target cell is a wall or outside map, cannot push
    if grid[target_r][target_c] == '#':
        return False, boxes

    # If the target cell is another box, we must consider pushing further, but this is handled iteratively by chain logic above.
    # Since we ended when we hit a non-box cell, we know it's not a box here.

    # If we reach here, we can push all boxes one step forward.
    new_boxes = set(boxes)
    # Push from the farthest box to the nearest box to avoid overwriting
    for (br, bc) in reversed(chain):
        new_boxes.remove((br, bc))
        new_boxes.add((br+dr, bc+dc))

    return True, new_boxes

def simulate(grid, moves):
    # grid is a list of list of chars
    # moves is a single string of move instructions

    robot_pos, boxes = find_robot_and_boxes(grid)

    # Directions
    dir_map = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }

    for move in moves:
        dr, dc = dir_map[move]
        r, c = robot_pos
        nr, nc = r + dr, c + dc  # next robot position

        # Check what is at nr,nc
        cell = grid[nr][nc]
        if cell == '#':
            # wall - no move
            continue
        elif (nr, nc) in boxes:
            # need to push
            can_push, new_boxes = can_push_boxes(grid, boxes, nr, nc, dr, dc)
            if can_push:
                # Update boxes
                boxes = new_boxes
                # Move robot
                robot_pos = (nr, nc)
            else:
                # cannot push, no move
                continue
        else:
            # empty floor, just move
            robot_pos = (nr, nc)

    return robot_pos, boxes

def compute_gps_sum(boxes):
    # GPS coordinate = 100 * row + col
    return sum(100 * r + c for (r, c) in boxes)

# Main execution
warehouse_lines, move_lines = parse_input("input.txt")

# Convert warehouse_lines to grid
grid = [list(line) for line in warehouse_lines]

# Combine all move lines into one big move string, ignoring newlines
moves = "".join(move_lines)

# Simulate
_, final_boxes = simulate(grid, moves)

# Compute sum of GPS coordinates
result = compute_gps_sum(final_boxes)
print(result)

1526018


In [6]:
import numpy as np
import matplotlib.pyplot as plt

dir_dict = {'<': (0, -1), '^': (-1, 0), '>': (0, 1), 'v': (1, 0)}

def move2(map15, pos, move):
    new_map = map15.copy()
    valid = True
    if move in ['<', '>']:
        # Horizontal move
        if move == '<':
            dx = -1
            start = pos[1] - 1  # look left
            end = -1            # stop at left boundary
            step = -1
        else:
            dx = 1
            start = pos[1] + 1  # look right
            end = map15.shape[1]
            step = 1

        # Find how far we can move
        x_max = pos[1]
        for x in range(start, end, step):
            if map15[pos[0], x] == -1:  # Wall encountered
                valid = False
                break
            if map15[pos[0], x] == 0:   # Free space encountered
                x_max = x
                break

        if valid and x_max != pos[1]:
            # We can move
            new_map[pos[0], pos[1]] = 0
            if move == '<':
                # Shift everything between x_max and pos[1]
                new_map[pos[0], x_max:pos[1]] = map15[pos[0], x_max+1:pos[1]+1]
            else:
                # Shift everything between pos[1] and x_max
                new_map[pos[0], pos[1]+1:x_max+1] = map15[pos[0], pos[1]:x_max]
            map15 = new_map
            pos = (pos[0], pos[1] + dx)

    elif move in ['^', 'v']:
        # Vertical move
        if move == '^':
            dy = -1
        else:
            dy = 1

        # We'll simulate pushing boxes vertically
        # M will track all connected box parts that need to move
        M = np.full(map15.shape, False, dtype=bool)
        pos_list = [pos]
        valid = True

        # BFS/DFS-like approach to find chain of boxes to push
        while pos_list and valid:
            new_pos_list = []
            for P in pos_list:
                if 0 <= P[0] < map15.shape[0] and 0 <= P[1] < map15.shape[1]:
                    M[P[0], P[1]] = True
                    # Check the cell we want to move into
                    target_val = map15[P[0] + dy, P[1]]
                    if target_val == -1:
                        # Wall in the way
                        valid = False
                        break
                    elif target_val in [1, 2]:
                        # Another box part is encountered
                        new_pos_list.append((P[0] + dy, P[1]))
                        # If it's a box side '1', also consider right cell
                        # If it's '2', consider left cell
                        if target_val == 1:
                            new_pos_list.append((P[0] + dy, P[1] + 1))
                        elif target_val == 2:
                            new_pos_list.append((P[0] + dy, P[1] - 1))

            pos_list = new_pos_list

        if valid:
            # Move the selected tiles by one step in dy
            M2 = np.full(map15.shape, False, dtype=bool)
            if move == '^':
                M2[:-1, :] = M[1:, :]
            else:
                M2[1:, :] = M[:-1, :]

            new_map[M] = 0
            new_map[M2] = map15[M]
            map15 = new_map
            pos = (pos[0] + dy, pos[1])

    return map15, pos

# Read and parse input
with open("input.txt", "r") as f:
    lines = f.read().strip().split('\n')

# Separate map lines (with '#') and move lines
warehouse_lines = []
moves_lines = []
found_moves = False
for line in lines:
    if '#' in line:
        if not found_moves:
            warehouse_lines.append(line)
        else:
            # If move lines somehow contain '#' adjust logic if needed
            # For now assume they do not
            pass
    else:
        # no '#' means moves
        found_moves = True
        if line.strip():
            moves_lines.append(line)

moves = "".join(moves_lines)

# Convert warehouse map to numeric map
char_to_num = {'.':0, '#':-1, 'O':1, '@':3}
map_array = np.array([list(row) for row in warehouse_lines])

map15 = np.vectorize(char_to_num.get)(map_array)

# Scale horizontally by factor of 2
map15 = np.repeat(map15, 2, axis=1)

# Adjust boxes: If an 'O' was here, after duplication we have two cells.
# The left one should be '1' and the right one '2'. We've already assigned 'O'->1,
# so we only need to fix odd columns that are part of a box:
for x, y in zip(*np.where(map15 == 1)):
    if y % 2 != 0:
        map15[x, y] = 2

# Find robot position (with '3')
robot_positions = np.where(map15 == 3)
if len(robot_positions[0]) == 0:
    raise ValueError("No robot found in map.")
pos = (robot_positions[0][0], robot_positions[1][0])

# The robot occupies one tile (3) and the next tile is scaled floor.
# Ensure next tile to the right is 0 if it exists and is currently not 0
# Actually, in the scaling step, '@' becomes '@.' mapped to (3,0) already.
# But if we initially read '@' as 3 and scaled by repetition, we have (3,3).
# Let's ensure second part of robot tile is floor:
if pos[1]+1 < map15.shape[1] and map15[pos[0], pos[1]+1] == 3:
    map15[pos[0], pos[1]+1] = 0

# Perform moves
for m in moves:
    map15, pos = move2(map15, pos, m)

# Now compute final GPS sum
# GPS uses the position of tile '1' (left half of the box)
boxes_positions = np.where(map15 == 1)
gps_sum = sum([100*x + y for x, y in zip(boxes_positions[0], boxes_positions[1])])
print(gps_sum)

1550677
